In [ ]:
import pandas as pd
import numpy as np
import gzip
import logging
import yaml
import codecs
global lcode
import re
import sys
from collections import Counter
from nltk import word_tokenize
import nltk
import spacy

import os
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import collections

#import tweepy as tw
from nltk.corpus import stopwords
import networkx

import warnings
warnings.filterwarnings("ignore")

sns.set(font_scale=1.5)
sns.set_style("whitegrid")

Corpora
http://www.nltk.org/nltk_data/

nltk.download() o python -m nltk.downloader all (consola)

In [ ]:
dateparse = lambda dates: [pd.datetime.strptime(d, '%Y-%m-%d %H:%M:%S') for d in dates]

In [ ]:
col_names = ["col1", "col2", "col3", "tweet", "via", "id", "followers", "following", "statuses", "loc", "col11", "col12", "col13",
            "col14", "col15", "col16", "col17", "col18", "col19", "col20", "col21", "col22", "col23", "col24", "col25",
            "col26","col27", "col28", "col29", "col30"]

df3 = pd.read_csv('/Users/cxb0266/Desktop/odio_politicas/streaming_odio_politicas_3.txt', sep= "\t",
                  names=col_names, encoding='utf-8')


In [ ]:
df0 = pd.read_csv('/Users/cxb0266/Desktop/odio_politicas/streaming_odio_politicas_0.txt', sep= "\t",
                  names=col_names, encoding='utf-8')

In [ ]:
df1 = pd.read_csv('/Users/cxb0266/Desktop/odio_politicas/streaming_odio_politicas_1.txt', sep= "\t",
                  names=col_names, encoding='utf-8')

In [ ]:
df2 = pd.read_csv('/Users/cxb0266/Desktop/odio_politicas/streaming_odio_politicas_2.txt', sep= "\t",
                  names=col_names, encoding='utf-8')

In [ ]:
df4 = pd.read_csv('/Users/cxb0266/Desktop/odio_politicas/streaming_odio_politicas_4.txt', sep= "\t",
                  names=col_names, encoding='utf-8')

In [ ]:
df5 = pd.read_csv('/Users/cxb0266/Desktop/odio_politicas/streaming_odio_politicas_5.txt', sep= "\t",
                  names=col_names, encoding='utf-8')

In [ ]:
df6 = pd.read_csv('/Users/cxb0266/Desktop/odio_politicas/streaming_odio_politicas_6.txt', sep= "\t",
                  names=col_names, encoding='utf-8')

In [ ]:
df7 = pd.read_csv('/Users/cxb0266/Desktop/odio_politicas/streaming_odio_politicas_7.txt', sep= "\t",
                  names=col_names, encoding='utf-8')

In [ ]:
df8 = pd.read_csv('/Users/cxb0266/Desktop/odio_politicas/streaming_odio_politicas_8.txt', sep= "\t",
                  names=col_names, encoding='utf-8')

In [ ]:
frames = [df0, df1, df2, df3, df4, df5, df6, df7, df8]
tweets=pd.concat(frames)

In [ ]:
tweets.info()

DataSets unidos. Tokenizar, palabras stop

In [ ]:
tweets=tweets.astype(str)

In [ ]:
pd.set_option('display.max_colwidth', -1)

In [ ]:
tweets.head()

In [ ]:
#Crear una nueva columa RT. Si es RT devulve 1 sino 0
new = tweets["tweet"].str.split("@", n = 1, expand = True) 

tweets["RT"]= new[0] 

tweets["tw"]= new[1]

In [ ]:
tweets['RT']=tweets['RT'].replace({'':0, 'RT ':1})

### Prueba: Numero de palabras en politicas_8

In [ ]:
words = Counter()
with open('/Users/cxb0266/Desktop/odio_politicas/streaming_odio_politicas_8.txt') as fin:
    words.update(word_tokenize(fin.read().lower()))

In [ ]:
print('There are ' + str(sum(words.values())) + ' total words.')

In [ ]:
print('There are ' + str(len(words)) + ' unique words.')

## Limpieza de datos

In [ ]:
def  clean_text(df):
    df = df.str.lower()
    df = df.apply(lambda elem: " ".join(re.sub("á", "a", elem).split()))
    df = df.apply(lambda elem: " ".join(re.sub("é", "e", elem).split()))
    df = df.apply(lambda elem: " ".join(re.sub("í", "i", elem).split()))
    df = df.apply(lambda elem: " ".join(re.sub("ó", "o", elem).split()))
    df = df.apply(lambda elem: " ".join(re.sub("ú", "u", elem).split()))
    df = df.apply(lambda elem: " ".join(re.sub("ñ", "n", elem).split()))
    df = df.apply(lambda elem: " ".join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|http.+?|^rt", " ", elem).split()))
    return df
tweets['tweet']=clean_text(tweets['tweet'])

Separar del tweet a quien va dirigido.

In [ ]:
newt = tweets["tw"].str.split(" ", n = 1, expand = True) 

tweets["Dir"]= newt[0] 

tweets["tw2"]= newt[1]

tweets.drop(columns =["tw"], inplace = True)
tweets.drop(columns =["tw2"], inplace = True)

Stop words

In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words_set = set(stopwords.words("spanish"))

In [ ]:
stop_words=list(stop_words_set)

Stop words sin acentos, hemos quitado los acentos de todos los tweets. Se envita confusión por faltas de ortografía 

In [ ]:
stop_words_mak=['tiene','vuestra','habiendo','han','tendremos','tenemos','hubisteis','hayas',
 'habian','seas','muy','se','tienes','habidos','mia','tuviste','tengas','contra','tuvieseis',
 'habiais','he','estariamos','estuvieron','para','fui','has','al','era','tenidos','nada',
 'estados','tuvieran','teneis','sentidas','fueran','estare','en','sois','que','estuvieras',
 'tendras','tuvieses','estado','erais','sus','si','seais','estuvieramos','hubimos','tuyas',
 'esteis','quienes','ella','todos','sea','fueses','otra','tenido','mios','del','sean','tenia',
 'mas','tambien','fueseis','ni','vuestro','habiamos','fueras','tu','que','sentida','antes',
 'y','hube','tendrian','tu','estarian','este','tuvierais','os','tendre','no','fueramos','vosotras',
 'ti','tendrias','seran','habriais','esta','estos','lo','estuviesemos','hubieses','eres',
 'nuestra','hayamos','estaras','la','estamos','habria','somos','tendran','yo','habras','habra',
 'seria','hay','poco','tendriamos','por','el','pero','habrias','estuviste','sobre','otro',
 'estaran','habeis','esos','tuvieron','hayais','cuando','estadas','estuvo','tenias','estemos',
 'sintiendo','teniamos','habias','tuvo','hubiese','ha','vuestras','son','esto','nosotros',
 'estaba','como','habre','habreis','mucho','habrian','vosotros','eras','fue','sentid','sera',
 'mi','estas','ante','es','tuvieramos','mias','me','hubieras','tenian','esta','estuvieses',
 'hubiesemos','tuve','fuese','estuvierais','los','hubieron','desde','fuiste','cual','estuve',
 'tus','mis','ya','nuestros','tuviese','donde','su','eran','ese','serias','habidas','durante',
 'habran','tengamos','tuvimos','le','estaban','seamos','estarias','un','algo','tengan','sereis',
 'estabamos','tuviesemos','estara','hubieseis','serian','seriais','tenida','estabais','estuvimos',
 'estuviesen','mio','fueron','siente','ellos','e','suya','estuviese','estuvieran','nosotras',
 'estuvieseis','o','tienen','las','habriamos','estuviera','tengo','estando','muchos','tenidas',
 'tendra','estaremos','el','hubo','estaria','sere','a','vuestros','hubierais','suyo','tuvisteis',
 'otros','tuvieras','eramos','estes','habido','estoy','suyas','estabas','fuerais','algunas',
 'esten','soy','otras','tanto','estariais','estareis','hasta','habremos','esa','hayan','mi',
 'eso','quien','tened','estan','tuya','hubieran','entre','seriamos','unos','hubiera','porque',
 'algunos','estas','fuera','estar','habida','fuesen','tendreis','con','tengais','suyos',
 'este','estad','fuisteis','sentido','todo','hubieramos','estuvisteis','sin','seremos','tuyos',
 'de','uno','les','nuestro','sentidos','teniais','hubiesen','hubiste','tuviesen','esas','tendria',
 'seras','teniendo','estais','te','tuyo','fuimos','haya','fuesemos','habia','nuestras','tenga',
 'una','tendriais','tuviera','hemos','nos','ellas','estada']

In [ ]:
stop_words

In [ ]:
stop_words_mak

## Tokenización y eliminar stop words

In [ ]:
def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords.words('spanish'):
            new_words.append(word)
    return new_words

In [ ]:
#tokenizar tweets['tweet']
tokenized_tweet = tweets['tweet'].apply(lambda x: x.split())
tokenized_tweet.head()

In [ ]:
tokenized_tweet=remove_stopwords(tokenized_tweet)

In [ ]:
tokenized_tweet

The tweets above have some elements that you do not want in your word counts. For instance, URLs will not be analyzed in this lesson. You can remove URLs (links) using regular expressions accessed from the re package.

https://www.earthdatascience.org/courses/earth-analytics-python/using-apis-natural-language-processing-twitter/calculate-tweet-word-frequencies-in-python/

Palabras STOP. The Python package nltk, commonly used for text analysis, provides a list of “stop words” that you can use to clean your Twitter data.

In [ ]:
tweets_nsw = [[word for word in words_in if not word in stop_words_mak]
              for words_in in tokenized_tweet]

## Bag of words
No la utilizo, pero para más adelante, quizas le sacamos utilidad

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
bow_vectorizer = CountVectorizer(max_df=0.90, min_df=2, max_features=20)
# bag-of-words feature matrix
bow = bow_vectorizer.fit_transform(tweets['tweet'])

In [ ]:
print(bow)

### Frecuencias 

In [ ]:
#Mas comunes, Utilizar tokenized_tweet = tweets['tweet'].apply(lambda x: x.split())

all_words_nsw = list(itertools.chain(*tweets_nsw)) # tokenized_tweet

counts_nsw = collections.Counter(all_words_nsw)

counts_nsw.most_common(100)



In [ ]:
#https://www.analyticsvidhya.com/blog/2018/07/hands-on-sentiment-analysis-dataset-python/
#Grafico mas comunes
clean_tweets_nsw = pd.DataFrame(counts_nsw.most_common(15),
                             columns=['words', 'count'])

fig, ax = plt.subplots(figsize=(8, 8))

# Plot horizontal bar graph
clean_tweets_nsw.sort_values(by='count').plot.barh(x='words',
                      y='count',
                      ax=ax,
                      color="purple")

ax.set_title("Common Words Found in Tweets (Without Stop Words)")

plt.show()

### Filtrar por tweets dirigidos a politicos (dir) y tweets escritos por politicos (col3)

In [ ]:
tweets['Dir'].unique()

In [ ]:
tweets['col3'].unique()

In [ ]:
tweets.Dir.value_counts()

In [ ]:
tweets.col3.value_counts()

Filtrar por politico tanto en col3 como en Dir, ver cuantos escribe cada uno y cuantos recibe, ver las palabras más habituales. 
Como al cortar el tweet original a veces quedaba algún signo de puntuación los eliminamos.

In [ ]:
tweets=tweets.astype(str)
def  clean_text_2(df):
    df = df.str.lower()
    df = df.apply(lambda elem: " ".join(re.sub("([^0-9A-Za-z \t])", " ", elem).split()))
    return df
tweets['Dir']=clean_text_2(tweets['Dir'])

Filtramos los tweets dirigidos a los ditintos politicos y obtenemos las palabras más frecuentes.

In [ ]:
sanchezcastejon = tweets.loc[tweets['Dir'] == "sanchezcastejon"]
santi_abascal = tweets.loc[tweets['Dir'] == "santi abascal"]
pablo_iglesias = tweets.loc[tweets['Dir'] == "pablo iglesias"] 
albert_rivera = tweets.loc[tweets['Dir'] == "albert rivera"]                            
inesarrimadas = tweets.loc[tweets['Dir'] == "inesarrimadas"]                       
pablocasado = tweets.loc[tweets['Dir'] == "pablocasado"]                             
irenemontero = tweets.loc[tweets['Dir'] == "irene montero"]                             
carmencalvo = tweets.loc[tweets['Dir'] == "carmencalvo"]                          
cayetanaat = tweets.loc[tweets['Dir'] == "cayetanaat"]                               
monasterior = tweets.loc[tweets['Dir'] == "monasterior"]

Función que tokeniza y elimina las stop words

In [ ]:
def num_palabras(politico):
    tokenized_tweet = politico['tweet'].apply(lambda x: x.split())

    tokenized_tweet=remove_stopwords(tokenized_tweet)

    tweets_nsw = [[word for word in words_in if not word in stop_words_mak]
              for words_in in tokenized_tweet]

    all_words_nsw = list(itertools.chain(*tweets_nsw)) # tokenized_tweet

    counts_nsw = collections.Counter(all_words_nsw)

    count = counts_nsw.most_common(100)

    return count

num_palabras(sanchezcastejon)

In [ ]:
num_palabras(santi_abascal)

In [ ]:
num_palabras(pablo_iglesias)

In [ ]:
num_palabras(albert_rivera)

In [ ]:
num_palabras(inesarrimadas)

In [ ]:
num_palabras(pablocasado)

In [ ]:
num_palabras(irenemontero)

In [ ]:
num_palabras(carmencalvo)

In [ ]:
num_palabras(cayetanaat)

In [ ]:
num_palabras(monasterior)

Filtramos los tweets escritos por los distintos politicos y obtenemos el número de tweets escritos y las palabras más frecuentes.

In [ ]:
col_3_sanchezcastejon = tweets.loc[tweets['col3'] == "@sanchezcastejon"]
col_3_santi_abascal = tweets.loc[tweets['col3'] == "@Santi_ABASCAL"]
col_3_pablo_iglesias = tweets.loc[tweets['col3'] == "@Pablo_Iglesias_"] 
col_3_albert_rivera = tweets.loc[tweets['col3'] == "@Albert_Rivera"]                            
col_3_inesarrimadas = tweets.loc[tweets['col3'] == "@InesArrimadas"]                       
col_3_pablocasado = tweets.loc[tweets['col3'] == "@pablocasado_"]                             
col_3_irene_montero = tweets.loc[tweets['col3'] == "@Irene_Montero_"]                             
col_3_carmencalvo = tweets.loc[tweets['col3'] == "@carmencalvo_"]                          
col_3_cayetanaat = tweets.loc[tweets['col3'] == "@cayetanaAT"]                               
col_3_monasterior = tweets.loc[tweets['col3'] == "@monasterioR"]

In [ ]:
print(col_3_sanchezcastejon.col3.count())
num_palabras(col_3_sanchezcastejon)

In [ ]:
print(col_3_santi_abascal.col3.count())
num_palabras(col_3_santi_abascal)

In [ ]:
print(col_3_pablo_iglesias.col3.count())
num_palabras(col_3_pablo_iglesias)

In [ ]:
print(col_3_albert_rivera.col3.count())
num_palabras(col_3_albert_rivera)

In [ ]:
print(col_3_inesarrimadas.col3.count())
num_palabras(col_3_inesarrimadas)

In [ ]:
print(col_3_pablocasado.col3.count())
num_palabras(col_3_pablocasado)

In [ ]:
print(col_3_irene_montero.col3.count())
num_palabras(col_3_irene_montero)

In [ ]:
print(col_3_carmencalvo.col3.count())
num_palabras(col_3_carmencalvo)

In [ ]:
print(col_3_cayetanaat.col3.count())
num_palabras(col_3_cayetanaat)

In [ ]:
print(col_3_monasterior.col3.count())
num_palabras(col_3_monasterior)

Filtrar los que son RT, obtener palabras más frecuentes tanto  RT en general, como por dirigido a politico y escrito por poltico. 

In [ ]:
RT = tweets.loc[tweets['RT'] == '1']

In [ ]:
num_palabras(RT)

Filtramos aquellos tweets que son RT DIRIGIDOS a politicos y obtenemos la frecuencia de palabras.

In [ ]:
RT_sanchezcastejon = RT.loc[RT['Dir'] == "sanchezcastejon"]
RT_santi_abascal = RT.loc[RT['Dir'] == "santi abascal"]
RT_pablo_iglesias = RT.loc[RT['Dir'] == "pablo iglesias"] 
RT_albert_rivera = RT.loc[RT['Dir'] == "albert rivera"]                            
RT_inesarrimadas = RT.loc[RT['Dir'] == "inesarrimadas"]                       
RT_pablocasado = RT.loc[RT['Dir'] == "pablocasado"]                             
RT_irenemontero = RT.loc[RT['Dir'] == "irene montero"]                             
RT_carmencalvo = RT.loc[RT['Dir'] == "carmencalvo"]                          
RT_cayetanaat = RT.loc[RT['Dir'] == "cayetanaat"]                               
RT_monasterior = RT.loc[RT['Dir'] == "monasterior"]

In [ ]:
num_palabras(RT_sanchezcastejon)

In [ ]:
num_palabras(RT_santi_abascal)

In [ ]:
num_palabras(RT_pablo_iglesias)

In [ ]:
num_palabras(RT_albert_rivera)

In [ ]:
num_palabras(RT_inesarrimadas)

In [ ]:
num_palabras(RT_pablocasado)

In [ ]:
num_palabras(RT_irenemontero)

In [ ]:
num_palabras(RT_carmencalvo)

In [ ]:
num_palabras(RT_cayetanaat)

In [ ]:
num_palabras(RT_monasterior)

Filtramos aquellos tweets que son RT ESCRITOS por politicos y obtenemos la frecuencia de palabras.

In [ ]:
RT_col_3_sanchezcastejon = RT.loc[RT['col3'] == "@sanchezcastejon"]
RT_col_3_santi_abascal = RT.loc[RT['col3'] == "@Santi_ABASCAL"]
RT_col_3_pablo_iglesias = RT.loc[RT['col3'] == "@Pablo_Iglesias_"] 
RT_col_3_albert_rivera = RT.loc[RT['col3'] == "@Albert_Rivera"]                            
RT_col_3_inesarrimadas = RT.loc[RT['col3'] == "@InesArrimadas"]                       
RT_col_3_pablocasado = RT.loc[RT['col3'] == "@pablocasado_"]                             
RT_col_3_irene_montero = RT.loc[RT['col3'] == "@Irene_Montero_"]                             
RT_col_3_carmencalvo = RT.loc[RT['col3'] == "@carmencalvo_"]                          
RT_col_3_cayetanaat = RT.loc[RT['col3'] == "@cayetanaAT"]                               
RT_col_3_monasterior = RT.loc[RT['col3'] == "@monasterioR"]

In [ ]:
num_palabras(RT_col_3_sanchezcastejon)

In [ ]:
num_palabras(RT_col_3_santi_abascal)

In [ ]:
num_palabras(RT_col_3_pablo_iglesias)

In [ ]:
num_palabras(RT_col_3_albert_rivera)

In [ ]:
num_palabras(RT_col_3_inesarrimadas)

In [ ]:
num_palabras(RT_col_3_pablocasado)

In [ ]:
num_palabras(RT_col_3_irene_montero)

In [ ]:
num_palabras(RT_col_3_carmencalvo)

In [ ]:
num_palabras(RT_col_3_cayetanaat)

In [ ]:
num_palabras(RT_col_3_monasterior)

Organizar tweets quincenalmente (hay más particiones), obtención de palabras más habituales en los distintos periodos tanto de tweets dirigidos como escritos a los distintos politicos.
Se puede observar la evolución.
Grupo por politico y por tiempo

In [ ]:
# Select observations between two datetimes
julio=tweets[(tweets['Fecha'] > '2019-07-05') & (tweets['Fecha'] <= '2019-08-05')]

In [ ]:
agosto=tweets[(tweets['Fecha'] > '2019-08-05') & (tweets['Fecha'] <= '2019-09-12')]

In [ ]:
Semana1=tweets[(tweets['Fecha'] > '2019-07-05') & (tweets['Fecha'] <= '2019-07-12')]

In [ ]:
Semana2=tweets[(tweets['Fecha'] > '2019-07-12') & (tweets['Fecha'] <= '2019-07-19')]

In [ ]:
Semana3=tweets[(tweets['Fecha'] > '2019-07-19') & (tweets['Fecha'] <= '2019-07-26')]

In [ ]:
Semana4=tweets[(tweets['Fecha'] > '2019-07-26') & (tweets['Fecha'] <= '2019-08-02')]

In [ ]:
Semana5=tweets[(tweets['Fecha'] > '2019-08-02') & (tweets['Fecha'] <= '2019-08-09')]

In [ ]:
Semana6=tweets[(tweets['Fecha'] > '2019-08-09') & (tweets['Fecha'] <= '2019-08-16')]

In [ ]:
Semana7=tweets[(tweets['Fecha'] > '2019-08-16') & (tweets['Fecha'] <= '2019-08-23')]

In [ ]:
Semana8=tweets[(tweets['Fecha'] > '2019-08-23') & (tweets['Fecha'] <= '2019-08-30')]

In [ ]:
Semana9=tweets[(tweets['Fecha'] > '2019-08-30') & (tweets['Fecha'] <= '2019-09-06')]

In [ ]:
Semana10=tweets[(tweets['Fecha'] > '2019-09-06') & (tweets['Fecha'] <= '2019-09-12')]

In [ ]:
primera15=tweets[(tweets['Fecha'] > '2019-07-05') & (tweets['Fecha'] <= '2019-07-19')]

In [ ]:
segunda15=tweets[(tweets['Fecha'] > '2019-07-19') & (tweets['Fecha'] <= '2019-08-02')]

In [ ]:
tercera15=tweets[(tweets['Fecha'] > '2019-08-02') & (tweets['Fecha'] <= '2019-08-16')]

In [ ]:
cuarta15=tweets[(tweets['Fecha'] > '2019-08-16') & (tweets['Fecha'] <= '2019-08-30')]

In [ ]:
quinta15=tweets[(tweets['Fecha'] > '2019-08-30') & (tweets['Fecha'] <= '2019-09-12')]

In [ ]:
primera15_sanchezcastejon = primera15.loc[primera15['Dir'] == "sanchezcastejon"]
primera15_santi_abascal = primera15.loc[primera15['Dir'] == "santi abascal"]
primera15_pablo_iglesias = primera15.loc[primera15['Dir'] == "pablo iglesias"] 
primera15_albert_rivera = primera15.loc[primera15['Dir'] == "albert rivera"]                            
primera15_inesarrimadas = primera15.loc[primera15['Dir'] == "inesarrimadas"]                       
primera15_pablocasado = primera15.loc[primera15['Dir'] == "pablocasado"]                             
primera15_irenemontero = primera15.loc[primera15['Dir'] == "irene montero"]                             
primera15_carmencalvo = primera15.loc[primera15['Dir'] == "carmencalvo"]                          
primera15_cayetanaat = primera15.loc[primera15['Dir'] == "cayetanaat"]                               
primera15_monasterior = primera15.loc[primera15['Dir'] == "monasterior"]

In [ ]:
primera15_col_3_sanchezcastejon = primera15.loc[primera15['col3'] == "@sanchezcastejon"]
primera15_col_3_santi_abascal = primera15.loc[primera15['col3'] == "@Santi_ABASCAL"]
primera15_col_3_pablo_iglesias = primera15.loc[primera15['col3'] == "@Pablo_Iglesias_"] 
primera15_col_3_albert_rivera = primera15.loc[primera15['col3'] == "@Albert_Rivera"]                            
primera15_col_3_inesarrimadas = primera15.loc[primera15['col3'] == "@InesArrimadas"]                       
primera15_col_3_pablocasado = primera15.loc[primera15['col3'] == "@pablocasado_"]                             
primera15_col_3_irene_montero = primera15.loc[primera15['col3'] == "@Irene_Montero_"]                             
primera15_col_3_carmencalvo = primera15.loc[primera15['col3'] == "@carmencalvo_"]                          
primera15_col_3_cayetanaat = primera15.loc[primera15['col3'] == "@cayetanaAT"]                               
primera15_col_3_monasterior = primera15.loc[primera15['col3'] == "@monasterioR"]

In [ ]:
segunda15_sanchezcastejon = segunda15.loc[segunda15['Dir'] == "sanchezcastejon"]
segunda15_santi_abascal = segunda15.loc[segunda15['Dir'] == "santi abascal"]
segunda15_pablo_iglesias = segunda15.loc[segunda15['Dir'] == "pablo iglesias"] 
segunda15_albert_rivera = segunda15.loc[segunda15['Dir'] == "albert rivera"]                            
segunda15_inesarrimadas = segunda15.loc[segunda15['Dir'] == "inesarrimadas"]                       
segunda15_pablocasado = segunda15.loc[segunda15['Dir'] == "pablocasado"]                             
segunda15_irenemontero = segunda15.loc[segunda15['Dir'] == "irene montero"]                             
segunda15_carmencalvo = segunda15.loc[segunda15['Dir'] == "carmencalvo"]                          
segunda15_cayetanaat = segunda15.loc[segunda15['Dir'] == "cayetanaat"]                               
segunda15_monasterior = segunda15.loc[segunda15['Dir'] == "monasterior"]

In [ ]:
segunda15_col_3_sanchezcastejon = segunda15.loc[segunda15['col3'] == "@sanchezcastejon"]
segunda15_col_3_santi_abascal = segunda15.loc[segunda15['col3'] == "@Santi_ABASCAL"]
segunda15_col_3_pablo_iglesias = segunda15.loc[segunda15['col3'] == "@Pablo_Iglesias_"] 
segunda15_col_3_albert_rivera = segunda15.loc[segunda15['col3'] == "@Albert_Rivera"]                            
segunda15_col_3_inesarrimadas = segunda15.loc[segunda15['col3'] == "@InesArrimadas"]                       
segunda15_col_3_pablocasado = segunda15.loc[segunda15['col3'] == "@pablocasado_"]                             
segunda15_col_3_irene_montero = segunda15.loc[segunda15['col3'] == "@Irene_Montero_"]                             
segunda15_col_3_carmencalvo = segunda15.loc[segunda15['col3'] == "@carmencalvo_"]                          
segunda15_col_3_cayetanaat = segunda15.loc[segunda15['col3'] == "@cayetanaAT"]                               
segunda15_col_3_monasterior = segunda15.loc[segunda15['col3'] == "@monasterioR"]

In [ ]:
tercera15_sanchezcastejon = tercera15.loc[tercera15['Dir'] == "sanchezcastejon"]
tercera15_santi_abascal = tercera15.loc[tercera15['Dir'] == "santi abascal"]
tercera15_pablo_iglesias = tercera15.loc[tercera15['Dir'] == "pablo iglesias"] 
tercera15_albert_rivera = tercera15.loc[tercera15['Dir'] == "albert rivera"]                            
tercera15_inesarrimadas = tercera15.loc[tercera15['Dir'] == "inesarrimadas"]                       
tercera15_pablocasado = tercera15.loc[tercera15['Dir'] == "pablocasado"]                             
tercera15_irenemontero = tercera15.loc[tercera15['Dir'] == "irene montero"]                             
tercera15_carmencalvo = tercera15.loc[tercera15['Dir'] == "carmencalvo"]                          
tercera15_cayetanaat = tercera15.loc[tercera15['Dir'] == "cayetanaat"]                               
tercera15_monasterior = tercera15.loc[tercera15['Dir'] == "monasterior"]

In [ ]:
tercera15_col_3_sanchezcastejon = tercera15.loc[tercera15['col3'] == "@sanchezcastejon"]
tercera15_col_3_santi_abascal = tercera15.loc[tercera15['col3'] == "@Santi_ABASCAL"]
tercera15_col_3_pablo_iglesias = tercera15.loc[tercera15['col3'] == "@Pablo_Iglesias_"] 
tercera15_col_3_albert_rivera = tercera15.loc[tercera15['col3'] == "@Albert_Rivera"]                            
tercera15_col_3_inesarrimadas = tercera15.loc[tercera15['col3'] == "@InesArrimadas"]                       
tercera15_col_3_pablocasado = tercera15.loc[tercera15['col3'] == "@pablocasado_"]                             
tercera15_col_3_irene_montero = tercera15.loc[tercera15['col3'] == "@Irene_Montero_"]                             
tercera15_col_3_carmencalvo = tercera15.loc[tercera15['col3'] == "@carmencalvo_"]                          
tercera15_col_3_cayetanaat = tercera15.loc[tercera15['col3'] == "@cayetanaAT"]                               
tercera15_col_3_monasterior = tercera15.loc[tercera15['col3'] == "@monasterioR"]

In [ ]:
cuarta15_sanchezcastejon = cuarta15.loc[cuarta15['Dir'] == "sanchezcastejon"]
cuarta15_santi_abascal = cuarta15.loc[cuarta15['Dir'] == "santi abascal"]
cuarta15_pablo_iglesias = cuarta15.loc[cuarta15['Dir'] == "pablo iglesias"] 
cuarta15_albert_rivera = cuarta15.loc[cuarta15['Dir'] == "albert rivera"]                            
cuarta15_inesarrimadas = cuarta15.loc[cuarta15['Dir'] == "inesarrimadas"]                       
cuarta15_pablocasado = cuarta15.loc[cuarta15['Dir'] == "pablocasado"]                             
cuarta15_irenemontero = cuarta15.loc[cuarta15['Dir'] == "irene montero"]                             
cuarta15_carmencalvo = cuarta15.loc[cuarta15['Dir'] == "carmencalvo"]                          
cuarta15_cayetanaat = cuarta15.loc[cuarta15['Dir'] == "cayetanaat"]                               
cuarta15_monasterior = cuarta15.loc[cuarta15['Dir'] == "monasterior"]

In [ ]:
cuarta15_col_3_sanchezcastejon = cuarta15.loc[cuarta15['col3'] == "@sanchezcastejon"]
cuarta15_col_3_santi_abascal = cuarta15.loc[cuarta15['col3'] == "@Santi_ABASCAL"]
cuarta15_col_3_pablo_iglesias = cuarta15.loc[cuarta15['col3'] == "@Pablo_Iglesias_"] 
cuarta15_col_3_albert_rivera = cuarta15.loc[cuarta15['col3'] == "@Albert_Rivera"]                            
cuarta15_col_3_inesarrimadas = cuarta15.loc[cuarta15['col3'] == "@InesArrimadas"]                       
cuarta15_col_3_pablocasado = cuarta15.loc[cuarta15['col3'] == "@pablocasado_"]                             
cuarta15_col_3_irene_montero = cuarta15.loc[cuarta15['col3'] == "@Irene_Montero_"]                             
cuarta15_col_3_carmencalvo = cuarta15.loc[cuarta15['col3'] == "@carmencalvo_"]                          
cuarta15_col_3_cayetanaat = cuarta15.loc[cuarta15['col3'] == "@cayetanaAT"]                               
cuarta15_col_3_monasterior = cuarta15.loc[cuarta15['col3'] == "@monasterioR"]

In [ ]:
quinta15_sanchezcastejon = quinta15.loc[quinta15['Dir'] == "sanchezcastejon"]
quinta15_santi_abascal = quinta15.loc[quinta15['Dir'] == "santi abascal"]
quinta15_pablo_iglesias = quinta15.loc[quinta15['Dir'] == "pablo iglesias"] 
quinta15_albert_rivera = quinta15.loc[quinta15['Dir'] == "albert rivera"]                            
quinta15_inesarrimadas = quinta15.loc[quinta15['Dir'] == "inesarrimadas"]                       
quinta15_pablocasado = quinta15.loc[quinta15['Dir'] == "pablocasado"]                             
quinta15_irenemontero = quinta15.loc[quinta15['Dir'] == "irene montero"]                             
quinta15_carmencalvo = quinta15.loc[quinta15['Dir'] == "carmencalvo"]                          
quinta15_cayetanaat = quinta15.loc[quinta15['Dir'] == "cayetanaat"]                               
quinta15_monasterior = quinta15.loc[quinta15['Dir'] == "monasterior"]

In [ ]:
quinta15_col_3_sanchezcastejon = quinta15.loc[quinta15['col3'] == "@sanchezcastejon"]
quinta15_col_3_santi_abascal = quinta15.loc[quinta15['col3'] == "@Santi_ABASCAL"]
quinta15_col_3_pablo_iglesias = quinta15.loc[quinta15['col3'] == "@Pablo_Iglesias_"] 
quinta15_col_3_albert_rivera = quinta15.loc[quinta15['col3'] == "@Albert_Rivera"]                            
quinta15_col_3_inesarrimadas = quinta15.loc[quinta15['col3'] == "@InesArrimadas"]                       
quinta15_col_3_pablocasado = quinta15.loc[quinta15['col3'] == "@pablocasado_"]                             
quinta15_col_3_irene_montero = quinta15.loc[quinta15['col3'] == "@Irene_Montero_"]                             
quinta15_col_3_carmencalvo = quinta15.loc[quinta15['col3'] == "@carmencalvo_"]                          
quinta15_col_3_cayetanaat = quinta15.loc[quinta15['col3'] == "@cayetanaAT"]                               
quinta15_col_3_monasterior = quinta15.loc[quinta15['col3'] == "@monasterioR"]

Como son muchas listas he reducido el número de palabras a 20.

In [ ]:
def num_palabras20(politico):
    tokenized_tweet = politico['tweet'].apply(lambda x: x.split())

    tokenized_tweet=remove_stopwords(tokenized_tweet)

    tweets_nsw = [[word for word in words_in if not word in stop_words_mak]
              for words_in in tokenized_tweet]

    all_words_nsw = list(itertools.chain(*tweets_nsw)) # tokenized_tweet

    counts_nsw = collections.Counter(all_words_nsw)

    count = counts_nsw.most_common(20)

    return count

In [ ]:
num_palabras20(primera15_sanchezcastejon)

In [ ]:
num_palabras20(segunda15_sanchezcastejon)

In [ ]:
num_palabras20(tercera15_sanchezcastejon)

In [ ]:
num_palabras20(cuarta15_sanchezcastejon)

In [ ]:
num_palabras20(quinta15_sanchezcastejon)

In [ ]:
num_palabras20(primera15_col_3_sanchezcastejon)

In [ ]:
num_palabras20(segunda15_col_3_sanchezcastejon)

In [ ]:
num_palabras20(tercera15_col_3_sanchezcastejon)

In [ ]:
num_palabras20(cuarta15_col_3_sanchezcastejon)

In [ ]:
num_palabras20(quinta15_col_3_sanchezcastejon)

In [ ]:
num_palabras20(primera15_santi_abascal)

In [ ]:
num_palabras20(segunda15_santi_abascal)

In [ ]:
num_palabras20(tercera15_santi_abascal)

In [ ]:
num_palabras20(cuarta15_santi_abascal)

In [ ]:
num_palabras20(quinta15_santi_abascal)

In [ ]:
num_palabras20(primera15_col_3_santi_abascal)

In [ ]:
num_palabras20(segunda15_col_3_santi_abascal)

In [ ]:
num_palabras20(tercera15_col_3_santi_abascal)

In [ ]:
num_palabras20(cuarta15_col_3_santi_abascal)

In [ ]:
num_palabras20(quinta15_col_3_santi_abascal)

In [ ]:
num_palabras20(primera15_pablo_iglesias)

In [ ]:
num_palabras20(segunda15_pablo_iglesias)

In [ ]:
num_palabras20(tercera15_pablo_iglesias)

In [ ]:
num_palabras20(cuarta15_pablo_iglesias)

In [ ]:
num_palabras20(quinta15_pablo_iglesias)

In [ ]:
num_palabras20(primera15_col_3_pablo_iglesias)

In [ ]:
num_palabras20(segunda15_col_3_pablo_iglesias)

In [ ]:
num_palabras20(tercera15_col_3_pablo_iglesias)

In [ ]:
num_palabras20(cuarta15_col_3_pablo_iglesias)

In [ ]:
num_palabras20(quinta15_col_3_pablo_iglesias)

In [ ]:
num_palabras20(primera15_albert_rivera)

In [ ]:
num_palabras20(segunda15_albert_rivera)

In [ ]:
num_palabras20(tercera15_albert_rivera)

In [ ]:
num_palabras20(cuarta15_albert_rivera)

In [ ]:
num_palabras20(quinta15_albert_rivera)

In [ ]:
num_palabras20(primera15_col_3_albert_rivera)

In [ ]:
num_palabras20(segunda15_col_3_albert_rivera)

In [ ]:
num_palabras20(tercera15_col_3_albert_rivera)

In [ ]:
num_palabras20(cuarta15_col_3_albert_rivera)

In [ ]:
num_palabras20(quinta15_col_3_albert_rivera)

In [ ]:
num_palabras20(primera15_inesarrimadas)

In [ ]:
num_palabras20(segunda15_inesarrimadas)

In [ ]:
num_palabras20(tercera15_inesarrimadas)

In [ ]:
num_palabras20(cuarta15_inesarrimadas)

In [ ]:
num_palabras20(quinta15_inesarrimadas)

In [ ]:
num_palabras20(primera15_col_3_inesarrimadas)

In [ ]:
num_palabras20(segunda15_col_3_inesarrimadas)

In [ ]:
num_palabras20(tercera15_col_3_inesarrimadas)

In [ ]:
num_palabras20(cuarta15_col_3_inesarrimadas)

In [ ]:
num_palabras20(quinta15_col_3_inesarrimadas)

In [ ]:
num_palabras20(primera15_pablocasado)

In [ ]:
num_palabras20(segunda15_pablocasado)

In [ ]:
num_palabras20(tercera15_pablocasado)

In [ ]:
num_palabras20(cuarta15_pablocasado)

In [ ]:
num_palabras20(quinta15_pablocasado)

In [ ]:
num_palabras20(primera15_col_3_pablocasado)

In [ ]:
num_palabras20(segunda15_col_3_pablocasado)

In [ ]:
num_palabras20(tercera15_col_3_pablocasado)

In [ ]:
num_palabras20(cuarta15_col_3_pablocasado)

In [ ]:
num_palabras20(quinta15_col_3_pablocasado)

In [ ]:
num_palabras20(primera15_irenemontero)

In [ ]:
num_palabras20(segunda15_irenemontero)

In [ ]:
num_palabras20(tercera15_irenemontero)

In [ ]:
num_palabras20(cuarta15_irenemontero)

In [ ]:
num_palabras20(quinta15_irenemontero)

In [ ]:
num_palabras20(primera15_col_3_irene_montero)

In [ ]:
num_palabras20(segunda15_col_3_irene_montero)

In [ ]:
num_palabras20(tercera15_col_3_irene_montero)

In [ ]:
num_palabras20(cuarta15_col_3_irene_montero)

In [ ]:
num_palabras20(quinta15_col_3_irene_montero)

In [ ]:
num_palabras20(primera15_carmencalvo)

In [ ]:
num_palabras20(segunda15_carmencalvo)

In [ ]:
num_palabras20(tercera15_carmencalvo)

In [ ]:
num_palabras20(cuarta15_carmencalvo)

In [ ]:
num_palabras20(quinta15_carmencalvo)

In [ ]:
num_palabras20(primera15_col_3_carmencalvo)

In [ ]:
num_palabras20(segunda15_col_3_carmencalvo)

In [ ]:
num_palabras20(tercera15_col_3_carmencalvo)

In [ ]:
num_palabras20(cuarta15_col_3_carmencalvo)

In [ ]:
num_palabras20(quinta15_col_3_carmencalvo)

In [ ]:
num_palabras20(primera15_cayetanaat)

In [ ]:
num_palabras20(segunda15_cayetanaat)

In [ ]:
num_palabras20(tercera15_cayetanaat)

In [ ]:
num_palabras20(cuarta15_cayetanaat)

In [ ]:
num_palabras20(quinta15_cayetanaat)

In [ ]:
num_palabras20(primera15_col_3_cayetanaat)

In [ ]:
num_palabras20(segunda15_col_3_cayetanaat)

In [ ]:
num_palabras20(tercera15_col_3_cayetanaat)

In [ ]:
num_palabras20(cuarta15_col_3_cayetanaat)

In [ ]:
num_palabras20(quinta15_col_3_cayetanaat)

In [ ]:
num_palabras20(primera15_monasterior)

In [ ]:
num_palabras20(segunda15_monasterior)

In [ ]:
num_palabras20(tercera15_monasterior)

In [ ]:
num_palabras20(cuarta15_monasterior)

In [ ]:
num_palabras20(quinta15_monasterior)

In [ ]:
num_palabras20(primera15_col_3_monasterior)

In [ ]:
num_palabras20(segunda15_col_3_monasterior)

In [ ]:
num_palabras20(tercera15_col_3_monasterior)

In [ ]:
num_palabras20(cuarta15_col_3_monasterior)

In [ ]:
num_palabras20(quinta15_col_3_monasterior)